# Optimization

Starting note: beware of *premature optimization*.  Don't worry about optimizing unless it's necessary.  

**However**, if you find your development cycle is dominated by waiting for your code to run, then consider optimization.  A bit of investment could save a lot of time and increase your overall productivity.

## Test problem:

Sum all squares up to some number $N - 1$:
$$ s = \sum_{i=0}^{N-1} i^2 $$

Let's start by writing down one way to do this in python.

In [ ]:
N = 1000000

def sum_baseline(N):
    s = 0
    for i in range(N):
        s += i**2
    return s

sum_baseline(N)

## Timing code

How fast does this execute?

In [ ]:
%timeit sum_baseline(N)

How different is this from computer to computer?  Anyone much slower or much faster?

## Examples

Example with list comprehension

In [ ]:
def sum_comprehension(N):
    return sum([i**2 for i in range(N)])

%timeit sum_baseline(N)
%timeit sum_comprehension(N)

Examples using numpy

In [ ]:
import numpy as np

def sum_numpy_good(N):
    a = np.arange(N)**2
    return a.sum()

def sum_numpy_bad(N):
    s = 0
    for i in np.arange(N):
        s += i**2 
    return s

In [ ]:
%timeit sum_baseline(N)
%timeit sum_numpy_good(N)
%timeit sum_numpy_bad(N)

General point: in python, avoid loops as much as possible when doing calculations!  That's what numpy is for.

A perhaps surprising/unexpected way to optimize?

In [ ]:
def sum_multiplication(N):
    s = 0 
    for i in range(N):
        s = i*i
    return s

%timeit sum_baseline(N)
%timeit sum_multiplication(N)

## Compiled examples 

1. cython
2. numba

### Cython: automatic python-to-C conversion

In [ ]:
# If necessary for you:
# !pip install cython

In [ ]:
%load_ext cython

In [ ]:
%%cython

def sum_cython(N):
    cdef int s = 0
    cdef int i
    for i in range(N):
        s += i**2
    return s

In [ ]:
%timeit sum_baseline(N)
%timeit sum_numpy_good(N)
%timeit sum_cython(N)

Side note: type declaration in cython really helps!

In [ ]:
%%cython

def sum_cython_slow(N):
    s = 0
    for i in range(N):
        s += i**2
    return s

In [ ]:
%timeit sum_cython(N)
%timeit sum_cython_slow(N)

### numba: "just-in-time" compilation

In [ ]:
# If you need to
#!pip install numba  

In [ ]:
from numba import jit

@jit(nopython=True)
def sum_numba(N):
    s = 0
    for i in range(N):
        s += i**2
    return s

In [ ]:
%timeit sum_numpy_good(N)
%timeit sum_cython(N)
%timeit sum_numba(N)

Advantage over cython?  Slightly easier development b/c *exact same* code runs in python; just add decorator.

## Profiling

The example calculation we have been working with is very simplified.  In real problems, in order to be able to optimize, you need to know where your execution bottlenecks are.  With this in mind, let's do a (very slightly) more complex example---reading in data from disk before summing it up.

First, let's make up the pretend data set.

In [ ]:
data = np.arange(N)
np.savetxt('pretend_data.txt', data)

Let's say we want to compute the sum of the squares of this dataset.  This is now a two-step process:

1. Read the data from disk.
2. Do the calculation.

In [ ]:
def my_pipeline():
    # load data
    data = np.loadtxt('pretend_data.txt')
    
    # compute sum
    s = 0
    for i in range(len(data)):
        s += i**2
    return s

In [ ]:
%timeit my_pipeline()

Finding the bottleneck by **profiling** your code.  Use the ipython "magic function" `%prun`:

In [ ]:
%prun my_pipeline()

### Line profiling

You can even do this line-by-line using the `line_profiler` package and Jupyter extension.

Install if you need to, and then load the Jupyter extension.

In [ ]:
# Install if you need to
# !pip install line_profiler

In [ ]:
%load_ext line_profiler

Now, you can use the `%lprun` magic function!  

In [ ]:
%lprun -f my_pipeline my_pipeline()

Reading in the data is clearly shown to be the bottleneck!

Let's see what we can do about that!

In [ ]:
# Save/load the data as binary instead of as a text file
np.save('pretend_data.npy', np.arange(N))

def my_binary_pipeline():
    data = np.load('pretend_data.npy')
    
    s = 0
    for i in range(len(data)):
        s += i**2
    return s

In [ ]:
%timeit my_binary_pipeline()

And now, line profiling:

In [ ]:
%lprun -f my_binary_pipeline my_binary_pipeline()

Now, the summing is indeed the bottleneck, so we can improve performance by optimizing code!

In [ ]:
def my_optimized_binary_pipeline():
    data = np.load('pretend_data.npy')
    return (data**2).sum()

In [ ]:
%timeit my_optimized_binary_pipeline()

In [ ]:
%lprun -f my_optimized_binary_pipeline my_optimized_binary_pipeline()

Great resource on profiling/timing code: 

https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html